
# Simple example of km3irf usage

This example produces *Effective Area* (Aeff), 
*Point Spread Function* (PSF), *Energy Dispersion* (Edisp) 
files in .fits format from `km3net_testdata`
simulation dst.root file.  
Then merge them into one common
.fits file. And finally these files can be read using `gammapy`


In [18]:
import km3irf
import gammapy

In [19]:
print(f"{gammapy.__name__}: {gammapy.__version__}")
print(f"{km3irf.__name__}: {km3irf.__version__}")

gammapy: 1.1
km3irf: 0.3.0


In [3]:
from km3net_testdata import data_path
# from km3irf import build_irf
from astropy.io import fits
# from km3irf.utils import merge_fits

import warnings

warnings.simplefilter("ignore")

Define a path to 10 events data file from ``km3net_testdata``



In [4]:
path_to_data = data_path("dst/mcv5.1.km3_numuCC.ALL.dst.bdt.10events.root")

## Effective Area
Create ``DataContainer`` object from test ``.dst`` file



In [7]:
test_irf = km3irf.build_irf.DataContainer(path_to_data)

Check, how many events in the file



In [8]:
test_irf.df.shape[0]

10

Apply default cuts, but they also can be changed by user



In [9]:
test_irf.apply_cuts()

Check, how many events survived after cuts:



In [10]:
test_irf.df.shape[0]

4

Apply re-weighting procedure for calculation new weights, which correspond
to a new spectral index. Weights are not needed for *Effective Area*.



In [11]:
weighted_dict = test_irf.weight_calc(tag="nu", df_pass=test_irf.df)
weighted_dict.values()

your input data file has no header, alpha_value set to default -1.4


dict_values([array([1.04317069, 0.95546222, 0.96131864, 1.04004845])])

To create ``.fits`` for *Effective area*, we need to pass a pandas data frame as input.



In [12]:
test_irf.build_aeff(df_pass=test_irf.df)

file aeff.fits is written successfully!


## Point Spread Function
In this part we create ``.fits`` for *Point Spread Function*



In [13]:
test_irf.build_psf(df_pass=test_irf.df)

file psf.fits is written successfully!


## Energy dispertion
In this part we create ``.fits`` for *Energy dispertion*



In [14]:
test_irf.build_edisp(norm=True, smooth=False, smooth_norm=False, df_pass=test_irf.df)

file edisp.fits is written successfully!


## Merge into one file
All ``.fits`` files can be merged in one ``.fits`` file for convenience.
Missing background file can be taken from data folder of the package.
Later the creation of background ``.fits`` file will be added.



In [15]:
km3irf.utils.merge_fits(
    aeff_fits="aeff.fits",
    psf_fits="psf.fits",
    edisp_fits="edisp.fits",
    output_path=".",
    output_file="new_IRF.fits",
)

combined IRF file new_IRF.fits is merged successfully!


## Check content with gammapy
Here is an example how to test created `.fits` files with [gammapy](https://gammapy.org/).  
`gammapy` should be installed.

In [32]:
aeff = gammapy.irf.EffectiveAreaTable2D.read("aeff.fits", hdu="EFFECTIVE AREA")
print(aeff)

EffectiveAreaTable2D
--------------------

  axes  : ['energy_true', 'offset']
  shape : (48, 12)
  ndim  : 2
  unit  : m2
  dtype : >f8



In [31]:
psf = gammapy.irf.PSF3D.read("psf.fits", hdu="PSF_2D_TABLE")
print(psf)

PSF3D
-----

  axes  : ['energy_true', 'offset', 'rad']
  shape : (24, 6, 111)
  ndim  : 3
  unit  : 1 / sr
  dtype : >f8



In [33]:
edisp = gammapy.irf.EnergyDispersion2D.read("edisp.fits", hdu="EDISP_2D")
print(edisp)

EnergyDispersion2D
------------------

  axes  : ['energy_true', 'migra', 'offset']
  shape : (24, 56, 6)
  ndim  : 3
  unit  : 
  dtype : >f8

